In [45]:
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D, AveragePooling2D,GlobalAveragePooling2D
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator

TRAINING_DATA = "datasets/train.csv"
TRAIN_COUNT = 2500
TEST_COUNT = 1678
FULL_SET = 4178
IMG_SIZE = (48,48)
images = []
target = []

In [46]:
#Read CSV to pandas DataFrame
data = pd.read_csv(TRAINING_DATA)
Emotions = data['Emotion'].values
Pixels = data['Pixels'].values

#Formatting pixels to Image Matrix
for i in range(len(Pixels)):
    pixelsFormatted = np.fromstring(Pixels[i], dtype = int, sep = " ").reshape(IMG_SIZE)
    images.append(pixelsFormatted)
    target.append(int(Emotions[i]))    

In [48]:
X_train = np.array(images[:3000])

Y_train = np.array(target[:3000])

#Adding Color depth
X_train = X_train.reshape(X_train.shape[0],48,48,1)
#7 classes of Output
Y_train = np_utils.to_categorical(Y_train,7)
#Converting every Element to float
X_train = X_train.astype('float32')
#Normalize
X_train /= 255

X_test = np.array(images[3000:])

Y_test = np.array(target[3000:])

X_test = X_test.reshape(X_test.shape[0],48,48,1)

Y_test = np_utils.to_categorical(Y_test,7)

X_test = X_test.astype('float32')

X_test /=255

In [56]:
model = Sequential()

#1st convolution layer
model.add(Convolution2D(64, (5, 5), activation='relu', input_shape=(48,48,1)))
model.add(MaxPooling2D(pool_size=(5,5), strides=(2, 2)))

#2nd convolution layer
model.add(Convolution2D(64, (3, 3), activation='relu'))
model.add(Convolution2D(64, (3, 3), activation='relu'))
model.add(AveragePooling2D(pool_size=(3,3), strides=(2, 2)))

#3rd convolution layer
model.add(Convolution2D(128, (3, 3), activation='relu'))
model.add(Convolution2D(128, (3, 3), activation='relu'))
model.add(AveragePooling2D(pool_size=(3,3), strides=(2, 2)))

model.add(Flatten())

#fully connected neural networks- connectes each input node to output node
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(7, activation='softmax'))

gen = ImageDataGenerator()
train_generator = gen.flow(X_train, Y_train, batch_size=64) 
test_generator = gen.flow(X_test,Y_test, batch_size=64)


model.compile(loss='categorical_crossentropy'
    , optimizer='adam'
    , metrics=['accuracy']
             )
callbacks = []
model.fit_generator(train_generator,steps_per_epoch = 100, epochs=5,validation_data=(X_test,Y_test) )

model.save('enjn.h5')

Epoch 1/5
100/100 [==============================] - 65s - loss: 1.8681 - acc: 0.2835 - val_loss: 1.8544 - val_acc: 0.2929
Epoch 2/5
100/100 [==============================] - 62s - loss: 1.8517 - acc: 0.2930 - val_loss: 1.8001 - val_acc: 0.3379
Epoch 3/5
100/100 [==============================] - 62s - loss: 1.6480 - acc: 0.4106 - val_loss: 1.4830 - val_acc: 0.4728
Epoch 4/5
100/100 [==============================] - 62s - loss: 1.3932 - acc: 0.4902 - val_loss: 1.3129 - val_acc: 0.5178
Epoch 5/5
100/100 [==============================] - 62s - loss: 1.2775 - acc: 0.5356 - val_loss: 1.2198 - val_acc: 0.5603


In [55]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_154 (Conv2D)          (None, 44, 44, 64)        1664      
_________________________________________________________________
max_pooling2d_26 (MaxPooling (None, 20, 20, 64)        0         
_________________________________________________________________
conv2d_155 (Conv2D)          (None, 18, 18, 64)        36928     
_________________________________________________________________
conv2d_156 (Conv2D)          (None, 16, 16, 64)        36928     
_________________________________________________________________
average_pooling2d_61 (Averag (None, 7, 7, 64)          0         
_________________________________________________________________
conv2d_157 (Conv2D)          (None, 5, 5, 128)         73856     
_________________________________________________________________
conv2d_158 (Conv2D)          (None, 3, 3, 128)         147584    
__________